Based on https://blog.devgenius.io/scraping-zillow-with-python-and-beautifulsoup-bbc7e581c218
and https://medium.com/@carsonlloydd/web-scraping-house-listings-from-zillow-62a7714d55f7

In [29]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import ast

In [2]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
city = 'phoenix/'
url = 'https://www.zillow.com/homes/for_sale/'+city
num_of_pages=30
urls_dict={}
rs_dict={}

with requests.Session() as s:
    r = s.get(url, headers=req_headers)
    for i in range(1, num_of_pages):
        urls_dict['url{0}'.format(i)]='https://www.zillow.com/homes/for_sale/'+city+'/{0}_p/'.format(i)
        rs_dict['r{0}'.format(i)]=s.get(urls_dict['url{0}'.format(i)], headers=req_headers)    

In [4]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(r.content, 'html.parser')
soups_dict={}
for i in range(1, num_of_pages):
        soups_dict['soup{0}'.format(i)]= BeautifulSoup(rs_dict['r{0}'.format(i)].content, 'html.parser') 

In [5]:
new_soup={'soup0':soup}
new_soup.update(soups_dict)

In [127]:
#create empty dataframes
dfs_dict={}
for i in range(num_of_pages):
        dfs_dict['df{0}'.format(i)]=pd.DataFrame()

# Read info from beatifulsoup parse
for key in new_soup:
    idx=list(new_soup).index(key)
    idx
    for i in new_soup[key]:
        soup = new_soup[key]
        address = soup.find_all (class_= 'list-card-addr')
        price = list(soup.find_all (class_='list-card-price'))
        beds = list(soup.find_all("ul", class_="list-card-details"))
        details = soup.find_all ('div', {'class': 'list-card-details'})
        #home_type = soup.find_all ('div', {'class': 'list-card-footer'})
        last_updated = soup.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        coord_tags = soup.findAll('script', {'type' : 'application/ld+json'})
        
        t = 0
        p = list(range(20))
        zipcode = []
        lat = []
        long = []
        url = []
        for v in p:
            try:
                ex = coord_tags[t].text
                _dict = ast.literal_eval(ex)
                zipcode.append(_dict['address']['postalCode'])
                lat.append(_dict['geo']['latitude'])
                long.append(_dict['geo']['longitude'])
                url.append(_dict['url'])
                t = t + 1
            except:
                t = t + 1
                continue

        #create dataframe columns out of variables
        
        
        df=dfs_dict['df{0}'.format(idx)]
        df['prices']= price
        df['address'] = address
        df['beds'] = beds 
        if len(beds)>len(url):
            zipcode.append(0)
            lat.append(0)
            long.append(0)
            url.append(0)
        df['zipcode'] = zipcode
        #df['home_type']=home_type
        df['lat'] = lat 
        df['long'] = long
        df['url'] = url
        

#     #create empty url list
#     urls = []

#     #loop through url, pull the href and strip out the address tag
#     for link in new_soup[key].find_all("article"):
#         href = link.find('a',class_="list-card-link")
#         if href == None:
#             addresses = None
#         else:
#             addresses = href.find('address',class_="list-card-addr").get_text()
#         urls.append(addresses)
#         urls = [x for x in urls if x is not None]

#     #import urls into a links column
#     dfs_dict['df{0}'.format(idx)]['links']=urls
#     dfs_dict['df{0}'.format(idx)]['links'] = dfs_dict['df{0}'.format(idx)]['links'].astype('str')
dfs=pd.DataFrame()
for key in dfs_dict:
    dfs=dfs.append(dfs_dict[key],ignore_index = True) 

Cleaning

In [128]:
#convert columns to str
dfs['prices'] = dfs['prices'].astype('str')
dfs['address'] = dfs['address'].astype('str')
dfs['beds'] = dfs['beds'].astype('str')

#remove html tags
dfs['prices'] = dfs['prices'].replace('<div class="list-card-price">', ' ', regex=True)
dfs['address'] = dfs['address'].replace('<address class="list-card-addr">', ' ', regex=True)
dfs['prices'] = dfs['prices'].replace('</div>', ' ', regex=True)
dfs['address'] = dfs['address'].replace('</address>', ' ', regex=True)
dfs['prices'] = dfs['prices'].str.replace(r'\D', '')

#remove html tags from beds column 
dfs['beds'] = dfs['beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<ul class="list-card-details"><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace(' abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('Studio</li><li>', '0 ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace(' abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- House for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Condo for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Apartment for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Townhouse for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Home for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Home for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- New construction</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- Multi-family home for sale</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- New construction</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- New construction</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- New construction</li></ul>', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot / Land for sale</li></ul>', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('Studio</li><li class="">', ' ', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot / Land for sale</li></ul>', '|', regex=True)
dfs['beds'] = dfs['beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">- Lot / Land for sale</li></ul>', ' ', regex=True)


/var/folders/xt/yyq8xk6n10sgk51827nl4cpw0000gn/T/ipykernel_15918/3187431190.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dfs['prices'] = dfs['prices'].str.replace(r'\D', '')


In [129]:
#dfs.columns = ['prices','address','bed_bath_sqft','links']
#split beds column into beds, bath and sq_feet
dfs[['beds','baths','sq_feet']] = dfs.beds.str.split(pat='|',expand=True)
#remove commas from sq_feet and convert to float
dfs.replace(',','', regex=True, inplace=True)
#convert column to float
dfs['prices'] = dfs['prices'].astype('float'

# #drop nulls
# df = df[(df['prices'] != '') & (df['prices']!= ' ')]

In [155]:
dfs=dfs.drop(labels=[44],axis=0)

In [157]:
dfs['beds'] = dfs['beds'].astype('int64')
dfs['baths'] = dfs['baths'].astype('int64')
dfs['sq_feet'] = dfs['sq_feet'].astype('int64')

,prices,address,beds,zipcode,lat,long,url,baths,sq_feet
0,254900.0,15818 N 25th St UNIT 112 Phoenix AZ 85032,3,85032,33.629870,-112.028788,https://www.zillow.com/homedetails/15818-N-25t...,3,1275
1,254900.0,3848 N 3rd Ave UNIT 1030 Phoenix AZ 85013,2,85013,33.493887,-112.080620,https://www.zillow.com/homedetails/3848-N-3rd-...,2,989
2,384900.0,522 E Marco Polo Rd Phoenix AZ 85024,3,85024,33.666465,-112.066867,https://www.zillow.com/homedetails/522-E-Marco...,3,1348
3,533100.0,16653 S 24th Pl Phoenix AZ 85048,4,85048,33.294945,-112.031217,https://www.zillow.com/homedetails/16653-S-24t...,3,2197
4,364900.0,2418 E Chipman Rd Phoenix AZ 85040,3,85040,33.402602,-112.029109,https://www.zillow.com/homedetails/2418-E-Chip...,2,1338
...,...,...,...,...,...,...,...,...,...
229,364900.0,2418 E Chipman Rd Phoenix AZ 85040,3,85040,33.402602,-112.029109,https://www.zillow.com/homedetails/2418-E-Chip...,2,1338
230,322100.0,10411 N 9th St UNIT 1 Phoenix AZ 85020,2,85020,33.580618,-112.060193,https://www.zillow.com/homedetails/10411-N-9th...,2,976
231,354900.0,1327 W Wahalla Ln Phoenix AZ 85027,2,85027,33.667171,-112.088014,https://www.zillow.com/homedetails/1327-W-Waha...,2,1091
232,243100.0,2929 W Yorkshire Dr UNIT 1041 Phoenix AZ 85027,2,85027,33.662520,-112.122142,https://www.zillow.com/homedetails/2929-W-York...,2,1019


In [158]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 233
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   prices   230 non-null    float64
 1   address  230 non-null    object 
 2   beds     230 non-null    int64  
 3   zipcode  230 non-null    object 
 4   lat      230 non-null    float64
 5   long     230 non-null    float64
 6   url      230 non-null    object 
 7   baths    230 non-null    int64  
 8   sq_feet  230 non-null    int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 18.0+ KB


In [159]:
dfs.to_csv('zillow_listing_phx.csv', index=False)